In [1]:
import numpy as np
import cv2
import scipy
import matplotlib.pyplot as plt
import math
import os
from Utils.ImageUtils import *
from Utils.MiscUtils import *
from Utils.GeometryUtils import *
import random
  

# K = np.array([[  9.037596e+02, 0.00000000e+00,6.957519e+02],
#             [  0.00000000e+00,9.019653e+02,2.242509e+02],
#             [  0.00000000e+00,0.00000000e+00,1.00000000e+00]])

# #Distortion Coefficients
# dist = np.array([[ -3.639558e-01, 1.788651e-01, 6.029694e-04, -3.922424e-04, -5.382460e-02]])




In [2]:
# def getCamera(config_file_name):
#     config_file = open(config_file_name)
#     config_data = yaml.load(config_file, Loader=yaml.FullLoader)

#     K = config_data.get('K')
#     D = config_data.get('D')

#     K = np.array(K.split(), np.float32).reshape(3,3)
#     D = np.array(D.split(), np.float32)

#     return K, D


In [3]:
# import yaml
# config_file_name = "/home/sakshi/courses/ENPM673/sakshi_p2/Data/Project2_Dataset2/data_1/camera_params.yaml"
# config_file = open(config_file_name)
# config_data = yaml.load(config_file, Loader=yaml.FullLoader)

In [4]:
# K = config_data.get('K')
# D = config_data.get('D')

# K = np.array(K.split(), np.float32).reshape(3,3)
# D = np.array(D.split(), np.float32)

In [5]:
# K, D

In [6]:
folder_name = "/home/sakshi/courses/ENPM673/sakshi_p2/Data/Project2_Dataset2/data_1/data"
config_file_name = "/home/sakshi/courses/ENPM673/sakshi_p2/Data/Project2_Dataset2/data_1/camera_params.yaml"
images = loadImages(folder_name)
K, D = getCamera(config_file_name)

Loading images from  /home/sakshi/courses/ENPM673/sakshi_p2/Data/Project2_Dataset2/data_1/data


In [4]:
old_right_line = None
old_left_line = None
while images:
    image = images.pop(0)
    image_overlay = image.copy()

    h, w, _ = image.shape
    #undistort the image
    image_undistorted = cv2.undistort(image, K, D)

    #remove noise
    image_blur = cv2.GaussianBlur(image_undistorted, (5,5), 0)

    #get bin
    image_bin = extractWhite(image_blur, threshold = 250)

    #roi
    image_roi, cropped_image = getROI(image_bin, 0.45) 

    #extract points
    filtered_image_roi = sobelXYThresh(image_roi)

    #get lines
    linesP = cv2.HoughLinesP(image_roi, 1, np.pi / 180, 50, None, 50, 10)

    #select left and right lines
    right_line, left_line = filterLines(linesP, old_right_line, old_left_line)
    old_right_line = right_line
    old_left_line = left_line

    # extend line
    extended_right_line = extendLines(right_line,  0.55 * h,  h)
    extended_left_line = extendLines(left_line,  0.55 * h,  h)

    # fill polygon
    corners = []
    
    corners.append([extended_left_line[0], extended_left_line[1]])
    corners.append([extended_right_line[0], extended_right_line[1]])
    corners.append([extended_right_line[2], extended_right_line[3]])
    corners.append([extended_left_line[2], extended_left_line[3]])
    corners = np.array(corners)

    cv2.fillPoly(image_overlay, pts =[corners], color=(0,0,255))

    #overlay
    cv2.addWeighted(image_overlay, 0.4, image, 0.6,
    0, image)

    #draw lines    
    cv2.line(image, (extended_right_line[0], extended_right_line[1]), (extended_right_line[2], extended_right_line[3]), (0, 0, 255), 3, cv2.LINE_AA)
    cv2.line(image, (extended_left_line[0], extended_left_line[1]), (extended_left_line[2], extended_left_line[3]), (0, 0, 255), 3, cv2.LINE_AA)

    cv2.imshow('frame', image)
    cv2.waitKey(10)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

left angle =  -30.860891688349497
right angle =  48.81407483429035
left angle =  -30.96375653207352
right angle =  48.957553114127855
right angle =  49.2538364361192
left angle =  -32.09259132873416
right angle =  48.94518622903757
left angle =  -32.87626175221062
right angle =  48.094058058917106
left angle =  -31.74528742235523
right angle =  48.012787504183336
left angle =  -31.639781555235515
right angle =  45.54565759341572
right angle =  46.005086005254185
left angle =  -31.56181184164935
right angle =  47.337305859123816
left angle =  -32.855721950433
left angle =  -31.84516052865566
right angle =  45.51616422976484
left angle =  -32.69946112100191
right angle =  47.950779109006355
right angle =  45.53545898556454
left angle =  -31.7594800848128
right angle =  45.93919094573558
left angle =  -31.607502246248906
right angle =  45.89053625336058
left angle =  -32.005383208083494
right angle =  45.507029060914775
left angle =  -31.920876219933184
left angle =  -32.592581472599264
l

In [16]:
# image = images[10]
# h, w, _ = image.shape
# #undistort the image
# image_undistorted = cv2.undistort(image, K, dist)

# #remove noise
# image_blur = cv2.GaussianBlur(image_undistorted, (5,5), 0)

# #get bin
# image_bin = extractWhite(image_blur, threshold = 250)

# #roi
# image_roi, cropped_image = getROI(image_bin, 0.45) 

# #extract points
# filtered_image_roi = sobelXYThresh(image_roi)

# #get lines
# linesP = cv2.HoughLinesP(image_roi, 1, np.pi / 180, 50, None, 50, 10)

# #select left and right lines
# right_line, left_line = filterLinestest(linesP, old_right_line, old_left_line)
# old_right_line = right_line
# old_left_line = left_line

# # extend line

# #draw lines    
# c = extendLines(right_line,  0.45 * h,  h)
# d = extendLines(left_line,  0.45 * h,  h)
# cv2.line(image_blur, (c[0], c[1]), (c[2], c[3]), (0,random.randint(0, 255),random.randint(0,255)), 3, cv2.LINE_AA)
# cv2.line(image_blur, (d[0], d[1]), (d[2], d[3]), (0,random.randint(0, 255),random.randint(0,255)), 3, cv2.LINE_AA)

# cv2.imshow('frame', image_blur)
# cv2.waitKey()
# cv2.destroyAllWindows()

In [14]:
0.45 * h,  h

(230.4, 512)